In [ ]:


import pypfopt as ppo
from pypfopt import risk_models, expected_returns
import pandas as pd



# Parameters
trading_days = 252

# 1. Mean and Covariance from log returns
mu_daily = log_returns.mean()
cov_daily = log_returns.cov()

# 2. Annualized Expected Returns (linearized)
mu_annual_log = mu_daily * trading_days
mu_annual_lin = np.exp(mu_annual_log) - 1

# 3. Annualized Covariance
cov_annual = cov_daily * trading_days

# Umwandlung in lineare Kovarianzmatrix
mu_i = mu_annual_log.values.reshape(-1, 1)
mu_j = mu_annual_log.values.reshape(1, -1)

sigma_i = np.diag(cov_annual).reshape(-1, 1)
sigma_j = np.diag(cov_annual).reshape(1, -1)

# Exakte Transformation
cov_lin = (np.exp(cov_annual.values) - 1) * np.exp(mu_i + mu_j + 0.5 * (sigma_i + sigma_j))
cov_annual_lin = pd.DataFrame(cov_lin, index=log_returns.columns, columns=log_returns.columns)

std_dev_annual_lin = np.sqrt((cov_annual_lin))

std_dev_annual_lin_percent = std_dev_annual_lin * 100

In [ ]:
linearize = False
percentage = False
trading_days = 252

compounding = False
calculate_using_log_returns = True

mu_ppo = ppo.expected_returns.mean_historical_return(data, returns_data=False, 
                                                 compounding=compounding, 
                                                 log_returns=calculate_using_log_returns, 
                                                 frequency=trading_days)


mu_my = qf.calculate_expected_return(log_returns,
                                  returns_are_log=True, # Passt
                                  calculate_using_log_returns=calculate_using_log_returns, # Passt
                                  trading_days=trading_days, # Passt
                                  compounding=compounding, # Passt
                                  percentage=percentage)

print(f"Error: {np.abs(mu_my - mu_ppo).sum()}")

In [ ]:
linearize = False
percentage = False
trading_days = 252

compounding = True
calculate_using_log_returns = False

cov_ppo = ppo.risk_models.risk_matrix(data, method='sample_cov', compounding=compounding, log_returns=calculate_using_log_returns, frequency=trading_days)



cov_my = qf.calculate_covariance_of_return(returns=returns,
                                     returns_are_log=False, # Passt
                                     trading_days=trading_days,
                                     calculate_using_log_returns=calculate_using_log_returns, 
                                     linearize=linearize,
                                     percentage=percentage)

print(f"Error: {np.abs(cov_my - cov_ppo).sum().sum()}")

In [ ]:

cov = qf.calculate_covariance_of_return(returns=returns,
                                     returns_are_log=False, 
                                     trading_days=trading_days,
                                     calculate_using_log_returns=False, 
                                     linearize=False,
                                     as_std_matrix=False,
                                     percentage=False)



mu = qf.calculate_expected_return(returns=returns,
                                  returns_are_log=False, 
                                  calculate_using_log_returns=False, 
                                  trading_days=trading_days, 
                                  compounding=True, 
                                  percentage=False)


linearize = [False, True]
percentage = [False, True]
as_std_matrix = [False, True]
calculate_using_log_returns = [False, True]

for config in [(l, p, s, c) for l in linearize for p in percentage for s in as_std_matrix for c in calculate_using_log_returns]:
    linearize, percentage, as_std_matrix, calculate_using_log_returns = config     
    cov_my = qf.calculate_covariance_of_return(returns=returns,
                                     returns_are_log=False, 
                                     trading_days=trading_days,
                                     calculate_using_log_returns=calculate_using_log_returns, 
                                     linearize=linearize,
                                     as_std_matrix=as_std_matrix,
                                     percentage=percentage)

    qf.plot_risk_matrix(mu, cov_my,
                        title=f"Covariance Matrix with linearize={linearize}, percentage={percentage}, as_std_matrix={as_std_matrix}, calculate_using_log_returns={calculate_using_log_returns}",
                        figsize=(5, 10))

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import EfficientFrontier, risk_models, expected_returns
from pypfopt.plotting import plot_efficient_frontier


weight_bounds = (0.0, 1.0)  # Bounds for weights

df = data.copy()

# Remove the second level of the MultiIndex (the 'Close' indicator)
df.columns = df.columns.droplevel(1)

# Step 3: Calculate expected returns and covariance matrix
mu = expected_returns.mean_historical_return(df)  # Expected returns
S = risk_models.sample_cov(df)  # Covariance matrix



# Step 4: Create an efficient frontier object
ef = EfficientFrontier(mu, S, weight_bounds=weight_bounds)

# Step 5: Calculate key portfolios
# Max Sharpe Ratio Portfolio
ef_max_sharpe = EfficientFrontier(mu, S, weight_bounds=weight_bounds)
max_sharpe_weights = ef_max_sharpe.max_sharpe()
ret_ms, std_ms, sharpe_ms = ef_max_sharpe.portfolio_performance()

# Min Volatility Portfolio
ef_min_vol = EfficientFrontier(mu, S, weight_bounds=weight_bounds)
min_vol_weights = ef_min_vol.min_volatility()
ret_mv, std_mv, _ = ef_min_vol.portfolio_performance()

# Step 6: Plot Efficient Frontier
fig, ax = plt.subplots(figsize=(6, 6))

# Plot individual asset points in grayscale
for i, ticker in enumerate(df.columns):
    asset_return = mu[ticker]
    asset_vol = np.sqrt(S.loc[ticker, ticker])
    ax.scatter(asset_vol, asset_return, marker=".", s=70, color="0.2", label=ticker)
    ax.annotate(ticker, (asset_vol, asset_return),
                textcoords="offset points", xytext=(5, -2.5), ha="left", fontsize=9, color="0.2")

# Plot the efficient frontier
plot_efficient_frontier(ef, ax=ax, show_assets=False, color="1", linewidth=2)

# Step 7: Monte Carlo Simulation - Generate random portfolios
n_portfolios = 2000  # Number of random portfolios
mc_returns = []  # List to store portfolio returns
mc_vols = []  # List to store portfolio volatilities
mc_sharpes = []  # List to store Sharpe ratios

risk_free_rate = 0.0  # Risk-free rate (adjust as needed)

for _ in range(n_portfolios):
    # Generate random weights
    weights = np.random.dirichlet(np.ones(len(tickers)), size=1)[0]
    port_return = np.dot(weights, mu)  # Portfolio return
    port_vol = np.sqrt(np.dot(weights.T, np.dot(S, weights)))  # Portfolio volatility
    sharpe = (port_return - risk_free_rate) / port_vol if port_vol > 0 else np.nan  # Sharpe ratio
    mc_returns.append(port_return)
    mc_vols.append(port_vol)
    mc_sharpes.append(sharpe)

# Plot Monte Carlo portfolios, colored by Sharpe Ratio
cmap = "hsv"  # Colormap for Sharpe Ratio
sharpe_normalized = (mc_sharpes - np.nanmin(mc_sharpes)) / (np.nanmax(mc_sharpes) - np.nanmin(mc_sharpes))
sc = ax.scatter(mc_vols, mc_returns, c=mc_sharpes, cmap=cmap, s=5, alpha=0.6, label="Monte Carlo Portfolios")
cbar = plt.colorbar(sc, ax=ax, label="Sharpe Ratio", orientation='horizontal', location="bottom", shrink=1, aspect=40, anchor=(0.5, 0.5))

# Highlight optimal portfolios
ax.scatter(std_ms, ret_ms, marker="*", s=100, color="red", label="Max Sharpe Ratio")
ax.scatter(std_mv, ret_mv, marker=5, s=100, color="black", label="Min Volatility")

# Step 8: Customize plot
ax.set_xlabel("$\sigma_p = \sqrt{\mathbb{V}ar[r_p]}$ (Portfolio risk)")  # X-axis label
ax.set_ylabel("$\mathbb{E}[r_p]$ (Expected rate of return)")  # Y-axis label

# Legend above the plot
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc="upper center", bbox_to_anchor=(0.5, 1.3), ncol=4, frameon=True)
#ax.legend(loc="right", bbox_to_anchor=(0.5, 0.5, 0.1, 0.5), frameon=True)
# Grid and layout adjustments
plt.grid(True, color="0.85")
plt.tight_layout(rect=[0, 0, 1, 0.95])
#plt.xlim((0.3, 0.475))  # Adjust x-axis limits
#plt.ylim((-0.2, 0.6))  # Adjust y-axis limits

# Step 9: Save plot
#plt.savefig(os.path.join(script_dir, "efficient_frontier.png"), dpi=300, bbox_inches='tight')
#plt.savefig(os.path.join(script_dir, "efficient_frontier.pgf"), bbox_inches='tight')

# Show plot
plt.show()